In [ ]:
# default_exp core
# default_cls_lvl 3

In [ ]:
#hide
%load_ext line_profiler
%matplotlib widget

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Corefunctions
> Corefunctionality for data preparation of sequential data for pytorch, fastai models

# Application Structure
The data will be extracted and prepared via transforms. Those are grouped in:
- Type Transforms: Those extraxt the needed components from the source items, like input sequences or target scalar values. The work on single tensors.
- Item Transforms: Those Transforms may work on tuple level and therefore may process relationships between input and output.
- Batch Transform: Those transforms work on batch level. They receive batched tensors and may apply lazy transforms like normalization very effeciently.

An application example may look like the following:
- sourceitems: 
    - path extraction with hdf5 file endings
    - create pandas dataframe with information for type transforms, like slices
    - filter items in pandas dataframe
- type transforms: 
    - extract hdf5 input and output sequence
    - create windows
- item transforms: 
    - filter sequence by value
    - shift output sequence by 1 element
- batch transforms: 
    - noise injection
    - normalization
    

In [ ]:
#export
from fastai2.data.all import *
import h5py

In [ ]:
#export
def obj_in_lst(lst,cls):
    '''retrieve first object of type cls from a list'''
    return next(o for o in lst if type(o) is cls)

In [ ]:
#export
def count_parameters(model):
    '''retrieve number of trainable parameters of a model'''
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## 1. Extract Source Items
The file paths may be extracted with `get_files` of fastai2. `get_hdf_files` removes the need of writing the hdf5 file extension.

Then a pandas dataframe may be created in case further information for the source items need to be stored like slices for the windowing function.

### 1.1 Extract File Paths

In [ ]:
f_path = 'test_data/'
hdf_files = get_files(f_path,extensions='.hdf5',recurse=True)
len(hdf_files),hdf_files[0]

(3, Path('test_data/train/Sim_RealisticCycle1.hdf5'))

In [ ]:
#export
hdf_extensions = ['.hdf5']
def get_hdf_files(path,recurse=True, folders=None):
    "Get hdf5 files in `path` recursively, only in `folders`, if specified."
    return get_files(path, extensions=hdf_extensions, recurse=recurse, folders=folders)

In [ ]:
hdf_files = get_hdf_files(f_path)
len(hdf_files),hdf_files[0]

(3, Path('test_data/train/Sim_RealisticCycle1.hdf5'))

### 1.2 Create Source Dictionaries
In order to extract mulitple realizations of one file with different modifications, we create a list of properties. Pandas Dataframes are to slow for iteration but very fast and convenient for creations. So after creation of the pandas Dataframe we convert it to a list of dictionaries.

In [ ]:
#export
def apply_df_tfms(f_list,pd_tfms = None):
    '''Create Pandas Dataframe out of a list of items, with a list of df transforms applied'''
    df = pd.DataFrame(data=f_list.items,columns=['path'],dtype=str)
    if pd_tfms is not None:
        for t in pd_tfms:
            df = t(df)
    return df

In [ ]:
df = apply_df_tfms(hdf_files)
df.head()

,path
0,test_data/train/Sim_RealisticCycle1.hdf5
1,test_data/train/Sim_RealisticCycle2.hdf5
2,test_data/valid/Sim_RealisticCycle3.hdf5


In [ ]:
#export
def CreateDict(pd_tfms = None):
    '''Create List of Dictionarys out of a list of items, with a list of df transforms applied'''
    def _inner(f_list):
        return apply_df_tfms(f_list,pd_tfms).to_dict(orient='records')
    return _inner

In [ ]:
l_dict =CreateDict()(hdf_files)
l_dict

[{'path': 'test_data/train/Sim_RealisticCycle1.hdf5'},
 {'path': 'test_data/train/Sim_RealisticCycle2.hdf5'},
 {'path': 'test_data/valid/Sim_RealisticCycle3.hdf5'}]

In [ ]:
#export
def ValidClmContains(lst_valid):
    '''add validation column using a list of strings that are part of the validation frames'''
    def _inner(df):
        re_valid = '|'.join([re.escape(f) for f in lst_valid])
        df['valid'] = df.path.str.contains(re_valid)
        return df

    return _inner

In [ ]:
%%time
lst_valid = ['valid']
CreateDict([ValidClmContains(lst_valid)])(hdf_files)

CPU times: user 0 ns, sys: 2.95 ms, total: 2.95 ms
Wall time: 2.94 ms


[{'path': 'test_data/train/Sim_RealisticCycle1.hdf5', 'valid': False},
 {'path': 'test_data/train/Sim_RealisticCycle2.hdf5', 'valid': False},
 {'path': 'test_data/valid/Sim_RealisticCycle3.hdf5', 'valid': True}]

In [ ]:
#export
def ValidClmIs(lst_valid):
    '''adds validation column using a list of validation filenames'''
    def _inner(df):
        df['valid'] = df.path.isin([str(f) for f in lst_valid])
        return df

    return _inner

In [ ]:
%%time
lst_valid = ['test_data/train/Sim_RealisticCycle2.hdf5','test_data/valid/Sim_RealisticCycle3.hdf5']
CreateDict([ValidClmIs(lst_valid)])(hdf_files)

CPU times: user 1.84 ms, sys: 864 µs, total: 2.71 ms
Wall time: 2.68 ms


[{'path': 'test_data/train/Sim_RealisticCycle1.hdf5', 'valid': False},
 {'path': 'test_data/train/Sim_RealisticCycle2.hdf5', 'valid': True},
 {'path': 'test_data/valid/Sim_RealisticCycle3.hdf5', 'valid': True}]

In [ ]:
#export
def FilterClm(clm_name,func = lambda x:x):
    '''adds validation column using a list of validation filenames'''
    def _inner(df):
        return df[func(df[clm_name])]

    return _inner

In [ ]:
CreateDict([ValidClmIs(lst_valid),FilterClm('valid')])(hdf_files)

[{'path': 'test_data/train/Sim_RealisticCycle2.hdf5', 'valid': True},
 {'path': 'test_data/valid/Sim_RealisticCycle3.hdf5', 'valid': True}]

In [ ]:
#export
def DfHDFCreateWindows(win_sz,stp_sz, clm, fixed_start = False, fixed_end = False,keep_filter=None):
    '''create windows of sequences, splits sequence into multiple items'''
    def _inner(df):
        if fixed_start and fixed_end: raise Exception
            
        pd.options.mode.chained_assignment = None #every row is a reference so we need to suppress the warning messages while copying
            
        df['l_slc'] = None
        df['r_slc'] = None

        lst_df = [] #new dataframe for every row
        if keep_filter is not None:
            keep_idxs = ~df.path.astype(str).str.contains(keep_filter)
        
        
        for idx, row in (df if keep_filter is None else df[keep_idxs]).iterrows():
            with h5py.File(row.path,'r') as f:
                #TODO make clm optional
#                 if clm == '': 
#                     clm = list(f.keys())[0]
                f_len = max(f[clm].shape)

                n_win = ((f_len-win_sz)//stp_sz)+1
                tmp_df = df.iloc[[idx]*n_win]; #duplicate the row of the df multiple times by reference
                lst_idx = np.arange(n_win)

                
                if not fixed_start: tmp_df['l_slc'] = lst_idx*stp_sz
                if not fixed_end: tmp_df['r_slc'] = lst_idx*stp_sz + win_sz

                lst_df.append(tmp_df)
                
        
        pd.options.mode.chained_assignment = 'warn'
                
        if keep_filter is not None: 
            raw_df = df[~keep_idxs].copy()
            lst_df.append(raw_df)
        
        res_df = pd.concat(lst_df,sort=False)
        return res_df
    
    return _inner

In [ ]:
# %%time
create_win = DfHDFCreateWindows(win_sz=100,stp_sz=100,clm='current')
win_df = create_win(df)
win_df

,path,l_slc,r_slc
0,test_data/train/Sim_RealisticCycle1.hdf5,0,100
0,test_data/train/Sim_RealisticCycle1.hdf5,100,200
0,test_data/train/Sim_RealisticCycle1.hdf5,200,300
0,test_data/train/Sim_RealisticCycle1.hdf5,300,400
0,test_data/train/Sim_RealisticCycle1.hdf5,400,500
...,...,...,...
2,test_data/valid/Sim_RealisticCycle3.hdf5,265000,265100
2,test_data/valid/Sim_RealisticCycle3.hdf5,265100,265200
2,test_data/valid/Sim_RealisticCycle3.hdf5,265200,265300
2,test_data/valid/Sim_RealisticCycle3.hdf5,265300,265400


In [ ]:
query_expr = 'l_slc <= 200'
filt_df = win_df.query(query_expr)
filt_df

,path,l_slc,r_slc
0,test_data/train/Sim_RealisticCycle1.hdf5,0,100
0,test_data/train/Sim_RealisticCycle1.hdf5,100,200
0,test_data/train/Sim_RealisticCycle1.hdf5,200,300
1,test_data/train/Sim_RealisticCycle2.hdf5,0,100
1,test_data/train/Sim_RealisticCycle2.hdf5,100,200
1,test_data/train/Sim_RealisticCycle2.hdf5,200,300
2,test_data/valid/Sim_RealisticCycle3.hdf5,0,100
2,test_data/valid/Sim_RealisticCycle3.hdf5,100,200
2,test_data/valid/Sim_RealisticCycle3.hdf5,200,300


In [ ]:
#export
def DfFilterQuery(query):
    def _inner(df):
        return df.query(query)
    return _inner

In [ ]:
test_eq(DfFilterQuery(query_expr)(win_df),filt_df)

In [ ]:
%%time
tfm_src = CreateDict([ValidClmContains(['valid']),DfHDFCreateWindows(win_sz=100+1,stp_sz=10,clm='current')])
src_dicts = tfm_src(hdf_files)
src_dicts[:5]

CPU times: user 528 ms, sys: 4.73 ms, total: 533 ms
Wall time: 531 ms


[{'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 0,
  'r_slc': 101},
 {'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 10,
  'r_slc': 111},
 {'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 20,
  'r_slc': 121},
 {'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 30,
  'r_slc': 131},
 {'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 40,
  'r_slc': 141}]

## 2. Convert Paths to Sequence Objects
Der Pfad wird unter Angabe der Spaltennamen in Sequenzen und Skalare Werte umgewandelt, um so am Ende ein 3-Tupel zu erhalten aus:
- (Sequence, Scalar, Sequence) <-> (input,input,output)

### 2.1 Extract sequential data from hdf5-files
Two different functions, based on pandas df and on lists

#### 2.1.1 Shift time Series
Sometimes we need to shift columns of a sequence by a specific value. Then we cant simply slice the array but have to handle each column individually. First a performance test has to be made.

In [ ]:
#export
def calc_shift_offsets(clm_shift):
    clm_shift = array(clm_shift)
    l_offs = -min(clm_shift.min(),0)
    r_offs = -max(clm_shift.max(),0)
    l_shift = clm_shift+l_offs
    r_shift = clm_shift+r_offs
    dim_red = l_offs-r_offs
    return l_shift,r_shift,dim_red

In [ ]:
shft = [0,0,-1,1]
calc_shift_offsets(shft)

(array([1, 1, 0, 2]), array([-1, -1, -2,  0]), 2)

both shifting methods have their own performance character. vstack needs double the time on short sequences, while the creation of a seperate array with copy becomes worse starting at around 5000 elements

In [ ]:
# ta = array([[1,2,3]*2]*10000)

In [ ]:
# %%timeit
# y = np.vstack([ta[i:-ta.shape[1]+i,i] for i in range(ta.shape[1])]).T   

In [ ]:
# %%timeit
# x = np.zeros((ta.shape[0]-ta.shape[1],ta.shape[1]))
# for i in range(ta.shape[1]):
#     x[:,i] = ta[i:-ta.shape[1]+i,i]

#### 2.1.2 HDF2Sequence
HDF5 performance is massively affected by the dtype of the signals. f4 (32 bit floating point) Numbers are faster to load and lead to smaller files then f8 numbers.

In [ ]:
#export
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0,axis=0),axis=0) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [ ]:
#export
def downsample_mean(x,N):
    shp = x.shape
    trunc = -(x.shape[0] % N)
    trunc = trunc if trunc != 0 else None
    return x[:trunc,:].reshape((-1,N,x.shape[-1])).mean(axis=1)

In [ ]:
#export
def hdf_extract_sequence(hdf_path,clms,dataset = None, l_slc = None, r_slc= None):
    with h5py.File(hdf_path,'r') as f:
        ds = f if dataset is None else f[dataset]
        l_array = [ds[n][l_slc:r_slc] for n in clms]
        seq = np.stack(l_array,axis=-1)
        return seq

In [ ]:
#export
class Memoize:
    def __init__(self, fn):
        self.fn = fn
        self.memo = {}

    def __call__(self, *args):
        if args not in self.memo:
            self.memo[args] = self.fn(*args)
        return self.memo[args]

In [ ]:
#export
class HDF2Sequence(Transform):
    
    def __init__(self, clm_names,clm_shift=None,truncate_sz=None,to_cls=noop,cached=True):
        if not clm_shift is None:
            assert len(clm_shift)==len(clm_names) and all(isinstance(n, int) for n in clm_shift)
            self.l_shift,self.r_shift,_ = calc_shift_offsets(clm_shift)
        
        self._exseq = Memoize(self._hdf_extract_sequence) if cached else self._hdf_extract_sequence
        
        store_attr(self,'clm_names,clm_shift,truncate_sz,to_cls,cached')
        
    def _hdf_extract_sequence(self,hdf_path,dataset = None, l_slc = None, r_slc= None):
        with h5py.File(hdf_path,'r') as f:
            ds = f if dataset is None else f[dataset]
            l_array = [(ds[n][l_slc:r_slc]) for n in self.clm_names]
            seq = np.stack(l_array,axis=-1)
            return seq.astype('f8')#workaround for random bug, that mitigates convergence if the numpy array is an f4 array. Seems to make no sense because the result does not change.
    
    def _extract_dict_sequence(self,item):
        if isinstance(item,dict):
            path = item['path']
            dataset = item['dataset'] if 'dataset' in item else None
            l_slc = item['l_slc'] if 'l_slc' in item else None
            r_slc = item['r_slc'] if 'r_slc' in item else None

            if self.cached:
                seq = self._exseq(path,dataset,None,None)[l_slc:r_slc]
            else:
                seq = self._exseq(path,dataset,l_slc,r_slc)
        else:
            seq = self._exseq(str(item),None,None,None)

        #shift clms of result by given value 
        if not self.clm_shift is None:
            l_seq = seq.shape[0]
            seq = np.stack([seq[self.l_shift[i]:l_seq+self.r_shift[i],i] for i in range(seq.shape[1])],axis=-1)
            
        if not self.truncate_sz is None:
            seq = seq[truncate_sz:]
        
        #it is important to slice first and then do the class conversion
        return self.to_cls(seq) 

    def encodes(self, item)->None: 
        return self._extract_dict_sequence(item)

In [ ]:
# %%timeit
hdf2seq = HDF2Sequence(['current','voltage'],cached=False)
hdf2seq(hdf_files[0])

array([[ 0.        ,  4.18735027],
       [-0.1       ,  4.18935013],
       [-0.1       ,  4.18969536],
       ...,
       [ 8.83880043,  3.39321232],
       [ 8.84599972,  3.39287138],
       [ 8.85309982,  3.3925302 ]])

In [ ]:
hdf2seq = HDF2Sequence(['current','voltage'],clm_shift=[1,1])
hdf2seq(hdf_files[0])

array([[-0.1       ,  4.18935013],
       [-0.1       ,  4.18969536],
       [-0.1       ,  4.19003677],
       ...,
       [ 8.83880043,  3.39321232],
       [ 8.84599972,  3.39287138],
       [ 8.85309982,  3.3925302 ]])

In [ ]:
hdf2seq = HDF2Sequence(['current','voltage'],cached=False,to_cls=TensorSequencesInput)
hdf2seq(hdf_files[0])

TensorSequencesInput([[ 0.0000,  4.1874],
        [-0.1000,  4.1894],
        [-0.1000,  4.1897],
        ...,
        [ 8.8388,  3.3932],
        [ 8.8460,  3.3929],
        [ 8.8531,  3.3925]])

In [ ]:
%%timeit
hdf2seq(hdf_files[0])

24.1 ms ± 283 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
hdf2seq = HDF2Sequence(['current','voltage'],cached=True)

In [ ]:
# %%timeit
hdf2seq(hdf_files[0])

array([[ 0.        ,  4.18735027],
       [-0.1       ,  4.18935013],
       [-0.1       ,  4.18969536],
       ...,
       [ 8.83880043,  3.39321232],
       [ 8.84599972,  3.39287138],
       [ 8.85309982,  3.3925302 ]])

Die Funktion lässt sich mittels Pipeline auf eine Liste von Quellobjekten (hier Pfade) anwenden 

In [ ]:
hdf2seq = HDF2Sequence(['current'])
hdf2seq(hdf_files[0]).shape

(265607, 1)

In [ ]:
pipe = Pipeline(HDF2Sequence(['current','voltage']))

In [ ]:
# res_pipe = pipe(hdf_files)
# len(res_pipe), res_pipe[0][0]

In [ ]:
#export
def hdf2scalars(hdf_path,c_names):
    with h5py.File(hdf_path,'r') as f:
#         import pdb; pdb.set_trace()
#         l_array = [f[n][:][:,None] for n in c_names]
#         seq = np.concatenate(l_array,axis=1)
        return None

#### Performance Test
Caching stores the arrays for future use at every function call. Very usefull, especially for windows. Should allways be turned. Only explicitly turn it off when there is not enough memory for your data.

In [ ]:
tfms=[  [HDF2Sequence(['current','voltage'],cached=False)],
        [HDF2Sequence(['voltage'],cached=False)]]
dsrc = Datasets(src_dicts[:1000],tfms=tfms)

In [ ]:
len(dsrc)

1000

In [ ]:
# %%time
# for x in dsrc:
#     x

In [ ]:
tfms=[  [HDF2Sequence(['current','voltage'],cached=True,clm_shift=[1,2])],
        [HDF2Sequence(['voltage'],cached=True)]]
dsrc = Datasets(src_dicts[:1000],tfms=tfms)

In [ ]:
# %%timeit
for x in dsrc:
    x

Caching is way faster because every file gets loaded multiple times

### 2.1 Datatypes for Sequences and Scalars

In [ ]:
#export
class TensorSequences(TensorBase):#TensorBase
#     def __init__(self,x,c_names=None, **kwargs):
#         super().__init__()
#         self.c_names = c_names
    
    def show(self, ctx=None, **kwargs):
#         import pdb; pdb.set_trace()
        ax = ctx
        if ax is None: _,ax = plt.subplots()
        ax.plot(self)
#         if title is not None: ax.set_title(title)
        return ax

    @classmethod
    @delegates(HDF2Sequence, keep=True)
    def from_hdf(cls,clm_names,**kwargs):
        return HDF2Sequence(clm_names,**kwargs)
    
class TensorSequencesInput(TensorSequences): pass
class TensorSequencesOutput(TensorSequences): pass

In [ ]:
f = TensorSequencesInput.from_hdf(['current'])
type(f(hdf_files[0]))

numpy.ndarray

In [ ]:
# TensorSequences(np.ones((30,2))).show()

In [ ]:
#export
@Transform
def toTensorSequencesInput(o): return TensorSequencesInput(o)
@Transform
def toTensorSequencesOutput(o): return TensorSequencesOutput(o)

In [ ]:
#export
class TensorScalars(Tensor): pass
class TensorScalarsInput(TensorScalars): pass
class TensorScalarsOutput(TensorScalars): pass

## 3. Transformations


### 3.1 Sequence Slicing Transformation

In [ ]:
#export 
class SeqSlice(Transform):
    '''Take a slice from an array-like object. Useful for e.g. shifting input and output'''
    def __init__(self, l_slc=None,r_slc=None):
        self.l_slc,self.r_slc = l_slc,r_slc
        
    def encodes(self, o): return o[self.l_slc:self.r_slc]

In [ ]:
l_shift = SeqSlice(r_slc=-1)
arr = np.ones((5))
test_eq(l_shift(arr),arr[:-1])

### 3.2 Sequence Noise Injection Transformation

In [ ]:
#export
class SeqNoiseInjection(Transform):
    split_idx=0
    '''Adds normal distributed noise to the tensor sequence with seperate mean and std for every signal'''
    def __init__(self, std=1e-1,mean=0.):
        self.std,self.mean = tensor(std),tensor(mean)
        
    def setups(self, dl:DataLoader):
        #check the tensor type of your input
        #TODO: include scalar type case
        x,*_ = dl.one_batch()
        self.std = to_device(self.std,x.device)
        self.mean = to_device(self.mean,x.device)
        
    def encodes(self, o:TensorSequencesInput): 
        #expand creates a view on a tensor and is therefore very fast compared to copy
        return o+torch.normal(mean=self.mean.expand_as(o), 
                              std=self.std.expand_as(o))

In [ ]:
x = TensorSequencesInput(tensor([[1,1,1],[-1,-1,-1.0]]))
ns_mean = tensor([0.,10.1,3.1])
ns_std = tensor([1.,1.1,0.1])
x,x.shape

(TensorSequencesInput([[ 1.,  1.,  1.],
         [-1., -1., -1.]]),
 torch.Size([2, 3]))

In [ ]:
seq_noise = SeqNoiseInjection(std=ns_std,mean=ns_std)
seq_noise(x)

TensorSequencesInput([[ 1.,  1.,  1.],
        [-1., -1., -1.]])

In [ ]:
seq_noise = SeqNoiseInjection(std=ns_std*10)
seq_noise(x)

TensorSequencesInput([[ 1.,  1.,  1.],
        [-1., -1., -1.]])

### 3.2 Sequence Bias Injection Transformation

In [ ]:
#export
class SeqBiasInjection(Transform):
    split_idx=0
    '''Adds a normal distributed offset to the tensor sequence with seperate mean and std for every signal'''
    def __init__(self, std=1e-1,mean=0.):
        self.std,self.mean = tensor(std),tensor(mean)
        
    def setups(self, dl:DataLoader):
        #check the tensor type of your input
        #TODO: include scalar type case
        x,*_ = dl.one_batch()
        self.std = to_device(self.std,x.device)
        self.mean = to_device(self.mean,x.device)
        
    def encodes(self, o:TensorSequencesInput): 
        #expand creates a view on a tensor and is therefore very fast compared to copy
        mean=self.mean.repeat((o.shape[0],1,1)).expand((o.shape[0],1,o.shape[2]))
        std= self.std.repeat((o.shape[0],1,1)).expand((o.shape[0],1,o.shape[2]))
        n = torch.normal(mean=mean, std=std).expand_as(o)
        return o+n

In [ ]:
seq_bias = SeqBiasInjection(std=ns_std,mean=ns_std)
seq_bias(x)[...,0]

tensor([ 1., -1.])

In [ ]:
seq_bias.mean

tensor([1.0000, 1.1000, 0.1000])

In [ ]:
seq_bias = SeqBiasInjection(std=ns_std*10)
seq_bias(x)

TensorSequencesInput([[ 1.,  1.,  1.],
        [-1., -1., -1.]])

### 3.3 Normalization
`Normalize` is programmed for `TensorImage` as an input tensor. It gets. At init the variable axes need to be chosen correspondingly to the shape of your tensor.

In [ ]:
#export
@Normalize
def encodes(self, x:TensorSequencesInput): 
    return (x-self.mean) / self.std

@Normalize
def decodes(self, x:TensorSequencesInput):
    f = to_cpu if x.device.type=='cpu' else noop
    return (x*f(self.std) + f(self.mean))

In [ ]:
norm = Normalize.from_stats(mean=ns_mean,std=ns_std,dim=1,ndim=2,cuda=False)
x,norm(x)

(TensorSequencesInput([[ 1.,  1.,  1.],
         [-1., -1., -1.]]),
 TensorSequencesInput([[  1.0000,  -8.2727, -21.0000],
         [ -1.0000, -10.0909, -41.0000]]))

## 4. Split in Training, Validation
Splitting kann anhand von vorher bekannten Indizes, dem Dateipfad oder anderen allgemeinen Funktion durchgeführt werden.

Splitting innerhalb einer Sequenzen sollte in der Praxis nur dann geschehen wenn eine einzige Sequenz vorhanden ist. Diese kann dann vorher manuell geteilt werden.


### 4.1 Splitting mit vorgegebenem Index

In [ ]:
splitter = IndexSplitter([1,2])
test_eq(splitter(hdf_files),[[0],[1,2]])

### 4.2 Splitting mit allgemeiner Funktion
Items, bei denen die definierte Funktion `True` zurück gibt, werden den Validierungsdatensatz zugeordnet, der Rest dem Training. In diesem Fall wird nach dem Übergeordneten Ordnernamen gesucht.

In [ ]:
splitter = FuncSplitter(lambda o: Path(o).parent.name == 'valid')
test_eq(splitter(hdf_files),[[0,1],[2]])

### 4.3 Splitting anhand des Parent-Folders
Splitter, der Explizit Training und Validierungsordner den Datensätzen zuordnet

In [ ]:
#export
def _parent_idxs(items, name): return mask2idxs(Path(o).parent.name == name for o in items)

def ParentSplitter(train_name='train', valid_name='valid'):
    "Split `items` from the parent folder names (`train_name` and `valid_name`)."
    def _inner(o, **kwargs):
        return _parent_idxs(o, train_name),_parent_idxs(o, valid_name)
    return _inner

In [ ]:
splitter = ParentSplitter()
test_eq(splitter(hdf_files),[[0,1],[2]])

### 4.4 Percentage Splitter

In [ ]:
#export
def PercentageSplitter(pct=0.8):
    "Split `items` in order in relative quantity."
    def _inner(o, **kwargs):
        split_idx=int(len(o)*pct)
        return L(range(split_idx)),L(range(split_idx,len(o)))
    return _inner

In [ ]:
splitter = PercentageSplitter(0.7)
test_eq(splitter(hdf_files),[[0,1],[2]])

### 4.5 Apply To Dictionary
In Case of the Datablock API your items are a list of dictionaries. If you want to apply a Splitter to the path stored within you need a wrapper function.

In [ ]:
#export
def ApplyToDict(fn,key='path'):
    return lambda x:fn([i[key] for i in x])

### 4.6 Valid Column
Using the 'valid' column of the Dataframe that has been created by a transformation.

In [ ]:
#export
valid_clm_splitter =  FuncSplitter(lambda o:o['valid'])

In [ ]:
valid_clm_splitter(src_dicts)

((#53101) [0,1,2,3,4,5,6,7,8,9...],
 (#26550) [53101,53102,53103,53104,53105,53106,53107,53108,53109,53110...])

## 5. Dataloaders Creation
A Datasets combines all implemented components on item level.

In [ ]:
#export
def pad_sequence(batch):
    '''collate_fn for padding of sequences of different lengths, use in before_batch of databunch, still quite slow'''
    #takes list of tuples as input, returns list of tuples
    sorted_batch = sorted(batch, key=lambda x: x[0].shape[0], reverse=True)

    pad_func = partial(torch.nn.utils.rnn.pad_sequence,batch_first=True)
    padded_tensors = [pad_func([x[tup] for x in sorted_batch]) for tup in range(len(batch[0]))]
    padded_list = [retain_types(tuple([tup[entry] for tup in padded_tensors]),batch[0]) for entry in range(len(batch))]
    #retain types is important for decoding later back to source items
#     import pdb; pdb.set_trace()
    
    return padded_list

### 5.1 Low-Level with Transforms

In [ ]:
tfms=[  [HDF2Sequence(['current','voltage']),SeqSlice(l_slc=1),toTensorSequencesInput],
        [HDF2Sequence(['voltage']),SeqSlice(r_slc=-1),toTensorSequencesOutput]]
splits = splitter([x['path'] for x in src_dicts])
dsrc = Datasets(src_dicts,tfms=tfms,splits=splits)

In [ ]:
# %%timeit
# dsrc[0]

In [ ]:
db = dsrc.dataloaders(bs=128,after_batch=[SeqNoiseInjection(std=[1.1,0.01]),Normalize(axes=[0,1])],before_batch=pad_sequence)
db.one_batch()[0].shape

torch.Size([128, 100, 2])

### 5.2 Mid-Level with Datablock API

In [ ]:
#export
class SequenceBlock(TransformBlock):
    def __init__(self, seq_extract,padding=False):
        return super().__init__(type_tfms=[seq_extract],
                                batch_tfms=[Normalize(axes=[0,1])],
                                dls_kwargs={} if not padding else {'before_batch': pad_sequence})

    @classmethod
    @delegates(HDF2Sequence, keep=True)
    def from_hdf(cls, clm_names, seq_cls=TensorSequencesInput,padding=False, **kwargs):
        return cls(HDF2Sequence(clm_names,to_cls=seq_cls,**kwargs), padding)


In [ ]:
seq = DataBlock(blocks=(SequenceBlock.from_hdf(['current','voltage'],TensorSequencesInput,padding=True,cached=False),
                        SequenceBlock.from_hdf(['voltage'],TensorSequencesOutput,cached=False)),
                get_items=tfm_src,
                splitter=ApplyToDict(ParentSplitter()))

In [ ]:
db = seq.dataloaders(hdf_files)

In [ ]:
# _ = pickle.dumps(db)

### 5.3 High-Level with SequenceDataloaders

In [ ]:
#export
from torch.utils.data import Dataset,ConcatDataset
class Seq2SeqDS(Dataset):
    #Datensatz aus HDF5 Datei 

    def __init__(self, hdf_file, u_clms,y_clms, win_sz,stp_sz):
        super().__init__()
        store_attr(self,'u_clms,y_clms,win_sz,stp_sz')
        
        self.u = hdf_extract_sequence(hdf_file,u_clms)
        self.y = hdf_extract_sequence(hdf_file,y_clms)
        
#         if clm_shift is not None:
#             assert len(clm_shift[0])==len(u_clms) and len(clm_shift[1])==len(y_clms)
#             l_shift,r_shift,dim_red = calc_shift_offsets(clm_shift)
        
        self.len = (self.u.shape[0]-self.win_sz)//self.stp_sz or 1

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        if not (0 <= idx < len(self)): raise IndexError
#         import pdb; pdb.set_trace()
        i = idx*self.stp_sz + self.win_sz
        return (TensorSequencesInput(self.u[(i-self.win_sz+1):i+1]),
                TensorSequencesOutput(self.y[(i-self.win_sz+1):i+1]))

In [ ]:
#export
@delegates(TfmdDL, keep=True)
def Seq2SeqDataloaders(items,u_clms,y_clms, win_sz,stp_sz=1,splitter=None,**kwargs):
    if splitter is None:
        lst_items = [items]
    else:
        lst_items = [items[idx] for idx in splitter(items)]
        
    dss = [ConcatDataset([Seq2SeqDS(f,u_clms,y_clms, win_sz,stp_sz) for f in itms])
          for itms in lst_items]
    if not 'after_batch' in kwargs: kwargs['after_batch'] = [to_device,Normalize(axes=[0,1])]
    
    dls = [TfmdDL(ds,shuffle=(i == 0),**kwargs) for i,ds in enumerate(dss)]
    return DataLoaders(*dls)

In [ ]:
dls=Seq2SeqDataloaders(hdf_files,['current','voltage'],['voltage'],100,10,
                       splitter=ParentSplitter())

## 6. Show Batches and Results

In [ ]:
#export
def plot_sequence(axs,in_sig,targ_sig,out_sig=None,**kwargs):
    n_targ = targ_sig.shape[1]
    for j,ax in  enumerate(axs[:-1]):
        ax.plot(targ_sig[:,j])
        if out_sig is not None: 
            ax.plot(out_sig[:,j])
            ax.legend(['y','ŷ'])
            if 'ref' in kwargs:
                ax.plot(kwargs['ref'][:,j]) 
        ax.label_outer()
    axs[-1].plot(in_sig)

In [ ]:
#export
def plot_seqs_single_figure(n_samples,n_targ,samples,plot_func,outs=None,**kwargs):
    rows=max(1,((n_samples-1) // 3)+1)
    cols=min(3,n_samples)
    fig = plt.figure(figsize=(9,2*cols))
    outer_grid = fig.add_gridspec(rows, cols)
#     import pdb; pdb.set_trace()
    for i in range(n_samples):
        in_sig = samples[i][0]
        targ_sig = samples[i][1]
        if outs is not None: out_sig = outs[i][0]
        inner_grid = outer_grid[i].subgridspec(n_targ+1, 1)
        axs = [fig.add_subplot(inner_grid[j]) for j in range(n_targ+1)]
        plot_func(axs,in_sig,targ_sig,out_sig=out_sig if outs is not None else None,**kwargs)
    plt.tight_layout()

In [ ]:
#export
def plot_seqs_multi_figures(n_samples,n_targ,samples,plot_func,outs=None,**kwargs):
    for i in range(n_samples):
        fig = plt.figure(figsize=(9,3))
        axs = fig.subplots(nrows=n_targ+1,sharex=True)
        in_sig = samples[i][0]
        targ_sig = samples[i][1]
        if outs is not None:  out_sig = outs[i][0]
            
        plot_func(axs,in_sig,targ_sig,out_sig=out_sig if outs is not None else None,**kwargs)
        
        plt.tight_layout()

In [ ]:
#export
@typedispatch
def show_batch(x:TensorSequences, y:TensorSequences, samples, ctxs=None, max_n=6, **kwargs):
    n_samples = min(len(samples), max_n)
    n_targ = samples[0][1].shape[1]
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_sequence, **kwargs)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_sequence, **kwargs)
    return ctxs

In [ ]:
dls.show_batch(max_n=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#export
@typedispatch
def show_results(x:TensorSequences, y:TensorSequences, samples, outs, ctxs=None, max_n=2, **kwargs):
    n_samples = min(len(samples), max_n)
    n_targ = samples[0][1].shape[1]
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_sequence,outs, **kwargs)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_sequence,outs, **kwargs)
    return ctxs

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_model.ipynb.
Converted 02_learner.ipynb.
Converted 03_dataloaders.ipynb.
Converted 11_dualrnn.ipynb.
Converted 12_TensorQuaternions.ipynb.
Converted 13_HPOpt.ipynb.
Converted index.ipynb.
